## Set up google colab environment

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification/V1')

In [2]:
from sample_extraction_V1_SGCNN_7 import *
import scipy.io as sio

## Load Indian Pines Dataset - Source

In [3]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [4]:
data_source = uIndianPines['indian_pines_corrected']
ground_truth_source = gt_IndianPines['indian_pines_gt']

In [5]:
data_source.shape

(145, 145, 200)

In [6]:
ground_truth_source.shape

(145, 145)

# Load target dataset

In [7]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana_gt.mat')

In [8]:
data_target = uBotswana['Botswana']
ground_truth_target = gt_Botswana['Botswana_gt']

In [9]:
data_target.shape

(1476, 256, 145)

In [10]:
ground_truth_target.shape

(1476, 256)

## Distrubution of samples for each class in Source

In [11]:
class_distribution_source = pd.DataFrame(np.unique(ground_truth_source, return_counts = True))
class_distribution_source = class_distribution_source.transpose()
class_distribution_source.columns = ['class','samples']
class_distribution_source

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [12]:
classes_source , counts_source = np.unique(ground_truth_source, return_counts = True)
classes_source = classes_source[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes_source

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

# Class distribution of samples in Target

In [13]:
class_distribution_target = pd.DataFrame(np.unique(ground_truth_target, return_counts = True))
class_distribution_target = class_distribution_target.transpose()
class_distribution_target.columns = ['class','samples']
class_distribution_target

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [14]:
classes_target , counts_target = np.unique(ground_truth_target, return_counts = True)
classes_target = classes_target[1:] ## Dropping classes with small number of samples
classes_target

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with different overlap ratios and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [15]:
pretrain_source_models(training_set_size = [200],
                      classes = classes_source,
                      cube_size = 20,
                      overlap_ratio = 1,
                      data = data_source,
                      ground_truth = ground_truth_source,
                      batch_size = 20,
                      channels = 64,
                      epochs = 50,
                      Verbosity = 1,
                      accuracies = [],
                      learning_rate = 0.0001,
                      source_dataset = 'indian_pines')


Model training starts for data with 200 samples from each class in training set

Samples per class: [1368, 523, 323, 730, 356, 837, 2224, 460, 1085]
Total number of samples 7906.

Total number of samples in training set 1800.
unique classes in training set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in training set: [200 200 200 200 200 200 200 200 200]

Total number of samples in test set 6106.
unique classes in test set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in test set: [1168  323  123  530  156  637 2024  260  885]

X_train => (1800, 20, 20, 64)
X_test  => (6106, 20, 20, 64)
y_train => (1800, 9)
y_test  => (6106, 9)

Group convolution output:  (None, 20, 20, 64)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            

# Fine tune on botswana

In [16]:
transfer_results, confusion_matrixes = transfer_learning(source_dataset = 'indian_pines',
                                        target_dataset = 'botswana',
                                        data = data_target,
                                        ground_truth = ground_truth_target,
                                        training_samples_from_each_class = [15,30,45,60,75],
                                        source_training_size = [200],
                                        classes = classes_target,
                                        overlap_ratio = 1,
                                        channels = 64,
                                        cube_size = 20,
                                        learning_rate = 0.0001,
                                        epochs = 150,
                                        batch_size = 20,
                                        test_accuracies = [],
                                        )


Model training starts for data with 15 samples from each class in training set

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 20, 20, 64)   4160        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 20, 20, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 20, 20, 64)   0           batch_normalization[0][0]        
___________

# Transfer Learning results on Botswana

In [17]:
transfer_results

,Training samples per class finetuning,Training Samples per class pretraining,Training Samples,Test Samples,Test_Accuracies
0,15,200,210,2934,71.06
1,30,200,420,2724,71.06
2,45,200,630,2514,71.04
3,60,200,840,2304,71.04
4,75,200,1050,2094,68.93


# Classification accuracies per class for each model

In [18]:
confusion_matrixes[0]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,175,0,0,0,67,0,13,0,0,0,0,0,0,0,68.63
1,0,83,0,0,0,3,0,0,0,0,0,0,0,0,96.51
2,0,0,116,81,0,0,13,18,8,0,0,0,0,0,49.15
3,0,0,2,192,5,0,0,0,1,0,0,0,0,0,96.0
4,7,17,0,48,157,2,3,0,0,0,0,14,6,0,61.81
5,0,18,0,22,0,58,0,0,51,0,0,6,99,0,22.83
6,22,0,0,0,0,0,222,0,0,0,0,0,0,0,90.98
7,0,0,15,0,0,0,0,27,0,0,0,123,23,0,14.36
8,0,0,0,10,0,0,0,8,258,5,0,0,18,0,86.29
9,0,0,0,0,0,0,0,0,0,233,0,0,0,0,100.0


In [19]:
confusion_matrixes[1]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,159,0,0,0,80,0,1,0,0,0,0,0,0,0,66.25
1,0,71,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,107,78,0,0,5,21,10,0,0,0,0,0,48.42
3,0,0,0,181,0,0,0,0,4,0,0,0,0,0,97.84
4,11,7,0,55,148,0,0,3,0,0,0,15,0,0,61.92
5,0,17,0,23,0,44,0,0,42,0,0,5,108,0,18.41
6,21,0,0,0,0,0,208,0,0,0,0,0,0,0,90.83
7,0,0,12,0,0,0,0,22,0,0,0,121,18,0,12.72
8,0,0,0,0,0,0,0,12,256,3,0,0,13,0,90.14
9,0,0,0,0,0,0,0,0,0,218,0,0,0,0,100.0


In [20]:
confusion_matrixes[2]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,153,0,0,0,71,0,1,0,0,0,0,0,0,0,68.0
1,0,56,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,74,80,0,0,8,35,9,0,0,0,0,0,35.92
3,0,0,0,151,0,0,0,0,19,0,0,0,0,0,88.82
4,12,5,0,54,134,0,0,1,0,0,0,17,1,0,59.82
5,0,11,0,26,0,34,0,0,47,0,0,5,101,0,15.18
6,20,0,0,0,0,0,194,0,0,0,0,0,0,0,90.65
7,0,0,1,0,0,0,0,16,0,0,0,123,18,0,10.13
8,0,0,0,0,0,0,0,11,254,4,0,0,0,0,94.42
9,0,0,0,0,0,0,0,0,0,202,1,0,0,0,99.51


In [21]:
confusion_matrixes[3]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,164,0,0,0,45,0,1,0,0,0,0,0,0,0,78.1
1,0,28,0,0,0,13,0,0,0,0,0,0,0,0,68.29
2,0,0,92,82,0,0,2,9,6,0,0,0,0,0,48.17
3,0,0,0,121,0,0,0,0,34,0,0,0,0,0,78.06
4,13,0,0,46,131,2,0,2,0,0,0,15,0,0,62.68
5,0,0,0,40,0,22,0,0,40,0,0,6,101,0,10.53
6,20,0,0,0,0,0,179,0,0,0,0,0,0,0,89.95
7,0,0,0,0,0,0,0,3,0,0,0,125,15,0,2.1
8,0,2,0,0,0,0,0,15,219,3,0,0,15,0,86.22
9,0,0,0,0,0,0,0,0,0,182,6,0,0,0,96.81


In [22]:
confusion_matrixes[4]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,162,0,1,0,32,0,0,0,0,0,0,0,0,0,83.08
1,0,26,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,56,77,0,0,0,29,14,0,0,0,0,0,31.82
3,0,0,0,100,0,0,0,0,40,0,0,0,0,0,71.43
4,14,1,0,39,126,1,0,12,0,1,0,0,0,0,64.95
5,0,7,0,17,0,9,0,1,67,0,0,2,91,0,4.64
6,20,0,0,0,0,0,164,0,0,0,0,0,0,0,89.13
7,0,0,0,0,0,0,0,79,0,0,0,49,0,0,61.72
8,0,6,0,0,0,0,0,11,220,2,0,0,0,0,92.05
9,0,0,0,0,0,0,0,0,0,149,24,0,0,0,86.13
